# **Lecture 11. Time Series Analysis**

## 1. Definition and Importance of Time Series Analysis
- Time series data is a sequence of observations recorded at regular time intervals. This type of analysis is crucial for understanding underlying patterns to forecast future values.
- Applications include predicting stock market trends, weather forecasting, energy demand forecasting, and analyzing business metrics like sales and website traffic.

## 2. Components of Time Series
- **Trend**: Long-term movement in data over time, either up or down.
- **Seasonality**: Patterns that repeat at regular intervals, such as weekly, monthly, or quarterly.
- **Cyclical Patterns**: Fluctuations occurring at irregular intervals, influenced by economic or other external factors.
- **Irregularity (Noise)**: Random variation in the series.

## 3. Python Libraries Overview
- **pandas** for data manipulation and analysis.
- **NumPy** for numerical computing.
- **matplotlib** for plotting graphs.
- **statsmodels** for implementing statistical models.

## 4. Basics of Handling Time Series Data in Python

### Time Series Data Structures
- pandas `DateTimeIndex` for handling dates and times.
- Conversion functions like `pd.to_datetime()` for converting string dates to `datetime` objects.

### Indexing time series data using dates
```python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

filedir = '/content/drive/MyDrive/Teaching/FWE458_Spring2025/Lec9/'
fname = filedir + "COVID19-Historical-V2-ST.csv"

covid = pd.read_csv(fname)
covid = covid[["Date", "POS_NEW_CP"]]

# date index
covid['Date'] = pd.to_datetime(covid['Date'])
covid = covid.set_index('Date')

fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(covid.POS_NEW_CP, marker='.', linestyle='-', linewidth=0.5, label='daily')
ax.set_ylabel('New Positive Cases')
ax.legend()
```

### Slicing data to obtain specific time periods
``` python
covid = pd.read_csv(fname)
covid['Date'] = pd.to_datetime(covid['Date'])
covid = covid.set_index('Date')

# slicing data
covid = covid.loc['2020-04-01':'2021-06-15']

fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(covid.POS_NEW_CP, marker='.', linestyle='-', linewidth=0.5, label='daily')
ax.set_ylabel('New Positive Cases')
ax.legend()
```

### Aggregating data over time using `resample()`
``` python
covid = pd.read_csv(fname)
covid['Date'] = pd.to_datetime(covid['Date'])
covid = covid.set_index('Date')

fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(covid.POS_NEW_CP, marker='.', linestyle='-', linewidth=0.5, label='daily')
ax.plot(covid.POS_NEW_CP.resample('W').mean(),marker='o', markersize=8, linestyle='-', label='Weekly Resample')
ax.plot(covid.POS_NEW_CP.resample('M').mean(),marker='o', markersize=8, linestyle='-', label='Monthly Resample')
ax.plot(covid.POS_NEW_CP.resample('Y').mean(),marker='o', markersize=8, linestyle='-', label='Yearly Resample')

ax.set_ylabel('New Positive Cases')
ax.legend()
```

### Moving average
```python
# Moving average
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(covid.POS_NEW_CP, 'k.', label='daily')

# simple moving average (SMA)
ax.plot(covid.POS_NEW_CP.rolling(7).mean(), linestyle='-', linewidth=3, label='7-day moving average')

# Exponential Moving Average (EMA) - weights added
ax.plot(covid.POS_NEW_CP.ewm(7).mean(), linestyle='-', linewidth=3, label='20-day moving average')

# An Exponential Moving Average (EMA) is a weighted moving average that emphasizes recent data points more heavily than older ones by applying an exponentially decreasing weight over time. This makes the EMA more responsive to recent price changes or fluctuations in the data compared to a simple moving average (SMA). The calculation uses a smoothing factor, commonly defined as α = 2/(N + 1) for a chosen period N, which determines how quickly the weights decrease for older data.

# Cumulative Moving Average (EMA) - the width of the window increases as duration increases
ax.plot(covid.POS_NEW_CP.expanding(7).mean(), linestyle='-', linewidth=3, label='20-day moving average')

# A Cumulative Moving Average (CMA) calculates the average of all data points up to the current time, updating with each new observation by giving equal weight to all past values, which offers a stable overall trend but can be slow to reflect recent changes.

ax.set_ylabel('New Positive Cases')
ax.legend()
```

## **4. Time Series Analysis Techniques**

### Time series decomposition
Time series decomposition is a statistical method that deconstructs a time series into several components, typically including trend, seasonality, and residual (or irregular) components. This method is crucial for understanding the underlying patterns in the data, which can be particularly useful for forecasting. The `statsmodels` library in Python provides efficient tools for performing time series decomposition.

![picture](https://www.bounteous.com/sites/default/files/b_inline_20200914.png)

### Components of Time Series Decomposition

1. **Trend**: The trend component reflects the long-term progression of the series, showing an upward or downward movement over time. It represents the increase or decrease in the data's value over a long period.

2. **Seasonality**: Seasonality shows fluctuations that occur at specific regular intervals less than a year, such as monthly or quarterly. This component is useful for understanding repetitive patterns over fixed periods.

3. **Residual (Irregular or Noise)**: The residual component contains the randomness or irregular movements in the time series, not explained by the trend or seasonal components. It's essentially the remainder of the time series after the trend and seasonal components have been removed.

### Decomposition with statsmodels

`statsmodels` offers two main methods for time series decomposition: **additive** and **multiplicative**.

- **Additive Model**: Used when the seasonal variations are roughly constant over time. The observed time series is the sum of the trend, seasonal, and residual components.
  
  Yt = Tt+St+Rt

- **Multiplicative Model**: Used when the seasonal variations change proportionally to the level of the trend. The observed time series is the product of the trend, seasonal, and residual components.
  
  Yt = Tt\*St\*Rt

#### Examples of Time Series Decomposition Using statsmodels

```python
import statsmodels.api as sm

decomposition = sm.tsa.seasonal_decompose(covid.POS_NEW_CP, model='additive', period=365)
fig = decomposition.plot()
fig.set_size_inches(14,7)
```

```python
# do decomposition with the Maula Loa CO2 data
drive.mount('/content/drive')
filedir = '/content/drive/MyDrive/Teaching/FWE458_Spring2025/Lec11/'
fname = filedir + "co2_mm_mlo.csv"

#filedir = '/content/drive/MyDrive/Teaching/FWE458_Spring2025/Lec9/'
#fname = filedir + "COVID19-Historical-V2-ST.csv"

#covid = pd.read_csv(fname)
#covid = covid[["Date", "POS_NEW_CP"]]

co2 = pd.read_csv(fname, parse_dates= {"date" : ["year","month"]}, comment="#")
co2 = co2.set_index("date")
#co2

fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(co2.average, marker='.', linestyle='-', linewidth=0.5, label='daily')

decomposition = sm.tsa.seasonal_decompose(co2.average, model='additive', period=12)
fig = decomposition.plot()
fig.set_size_inches(14,7)

```

## Stationary vs Non-Stationary Time series

In time series analysis, distinguishing between stationary and non-stationary processes is fundamental. The concepts of stationarity and non-stationarity have profound implications for modeling and forecasting time series data.

### Stationary Time Series

A stationary time series is one whose properties do not depend on the time at which the series is observed. Thus, it doesn't matter when you observe a stationary series; its basic properties, like mean, variance, and autocorrelation, are constant over time. Stationarity is an important assumption in many time series analysis methods, as it implies that the time series is predictable and exhibits a regular structure over time.

**Characteristics of Stationary Time Series:**
- **Constant Mean:** The average value of the series does not change over time.
- **Constant Variance:** The variability of the series remains the same over time.
- **Constant Autocorrelation Structure:** How the series correlates with itself over different intervals does not change over time.
- **No Periodic Fluctuations (Seasonality):** While some stationary series might exhibit cyclic behavior, where the cycles are not of a fixed length, true seasonality violates stationarity.

**Testing for Stationarity:**
Tools like the Augmented Dickey-Fuller (ADF) test, KPSS test, and Phillips-Perron test are commonly used to statistically test for stationarity in a time series.

### Non-stationary Time Series

A non-stationary time series is one whose statistical properties change over time. This can manifest as a change in mean over time, a change in variance, or a change in other aspects of the series' distribution. Non-stationary data are more challenging to model and forecast due to their changing structure.

**Characteristics of Non-Stationary Time Series:**
- **Variable Mean:** The average value of the series may trend upwards or downwards over time.
- **Variable Variance:** The variability of the series may increase or decrease over time.
- **Variable Autocorrelation Structure:** The way in which the series' values are correlated with themselves over time may change.
- **Presence of Seasonality or Trends:** Non-stationary series often exhibit trends (either deterministic or stochastic) and/or seasonality.

Non-stationary time series analysis often involves making the series stationary before modeling. This is typically done through differencing the series, transforming the data (e.g., logarithmic transformation), or detrending.

### Why Stationarity Matters

Many statistical forecasting methods and models (like ARIMA) assume or require the time series to be stationary to make accurate predictions. This is because stationary series have a consistent structure that can be learned and predicted over time, whereas non-stationary series do not, making their future values more unpredictable with those models.

In practice, dealing with non-stationarity is a common task in time series analysis, and identifying whether a series is stationary or non-stationary is a critical first step in selecting the appropriate modeling approach.

#### An Everyday Analogy
Think of it like baking a cake with a recipe:

##### Stationary Data:
Imagine you have a fixed recipe. Every time you bake the cake, you use the same amount of flour, sugar, and eggs. Your cakes turn out similar because the ingredients and the process are consistent.

##### Non-Stationary Data:
Now, imagine if every time you baked a cake, someone changed the recipe—sometimes more sugar, sometimes less flour. Predicting the taste of the cake becomes much harder because the recipe keeps changing.

In time series analysis, stationarity means the "recipe" (the underlying process of the data) is consistent. Without it, you're trying to predict something with shifting rules, which is much more difficult and can lead to incorrect conclusions.



### Example for testing time series stationarity
```python
# ADF test
from statsmodels.tsa.stattools import adfuller

result = adfuller(co2.average)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1]) # if p> 0.05 Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.

#KPSS test
from statsmodels.tsa.stattools import kpss
result = kpss(co2.average)
print('KPSS Statistic: %f' % result[0])
print('p-value: %f' % result[1]) # if p < 0.05 non-stationary.
```

### Make your timeseries stationary
Making a time series stationary is a crucial step in time series analysis, especially for models that assume stationarity, such as ARIMA (Autoregressive Integrated Moving Average). A stationary time series has constant mean and variance over time, and its autocovariance does not depend on time. Here are several methods to transform a non-stationary time series into a stationary one:

#### 1. Differencing

Differencing is one of the most common methods to stabilize the mean of a time series by removing changes in the level of a time series, thus eliminating (or reducing) trend and seasonality.

- **First Differencing**: This involves subtracting the previous observation from the current observation. If the original time series is represented by $Y_t$, then the first-differenced series is $Y_t' = Y_t - Y_{t-1}$.
- **Seasonal Differencing**: If the series has a seasonal pattern, then seasonal differencing can be effective. This involves subtracting the observation from the same season in the previous cycle, e.g., $Y_t' = Y_t - Y_{t-n}$, where $n$ is the seasonality period.

#### 2. Transformation

Transformations such as logarithmic, square root, or power transformations can help stabilize the variance of a time series.

- **Log Transformation**: Applying a log transformation, $Y_t' = \log(Y_t)$, can help stabilize a growing variance.
 - Use a log transformation when your time series shows exponential growth or increasing variance, as it stabilizes the variance and linearizes multiplicative relationships, making the data more stationary.
- **Square Root Transformation**: The square root, $Y_t' = \sqrt{Y_t}$, can also be effective for stabilizing variance.
 - Use a square root transformation when your time series data involves count values or exhibits moderate heteroscedasticity, as it helps stabilize variance without compressing data as aggressively as a log transformation.
- **Box-Cox Transformation**: A more generalized approach that can stabilize variance and make the series more normal (Gaussian). The Box-Cox transformation requires a parameter, $\lambda$, and is defined as $Y_t'(\lambda) = \frac{Y_t^\lambda - 1}{\lambda}$ for $\lambda \neq 0$, and $Y_t' = \log(Y_t)$ for $\lambda = 0$.
 - Use a Box-Cox transformation when your strictly positive time series data exhibits heteroscedasticity or non-normality, as it finds the optimal power parameter to stabilize variance and approximate normality.

#### 3. Detrending

Detrending involves removing the underlying trend in the series. This can be achieved by:

- **Subtracting the Trend Line**: Fit a regression model to the time series and subtract the resulting trend line from the original series.
- **Moving Average**: Subtracting the moving average of a certain period can also help remove the trend, making the series more stationary.

#### 4. Decomposition

Time series decomposition involves separating the time series into trend, seasonal, and residual components. Once decomposed, you can reconstruct the series without the trend and/or seasonal components, potentially leaving a stationary residual component.

#### Example

```python
# detrending
y = co2.average
y_detrend = (y - y.rolling(window=12).mean())/y.rolling(window=12).std()

y_detrend.plot()
y_detrend.dropna(inplace=True)
result = ADF_test(y_detrend,'de-trended data')

# differencing

# This method removes the underlying seasonal or cyclical patterns in the time series.
y_12lag =  y - y.shift(12)
y_12lag.dropna(inplace=True)
y_12lag.plot()
ADF_test(y_12lag, '12 lag differenced data')

# Detrending + Differencing

y_12lag_detrend =  y_detrend - y_detrend.shift(12)
y_12lag_detrend.plot()
ADF_test(y_12lag_detrend,'12 lag differenced de-trended data')
```

Transforming a non-stationary time series into a stationary one is essential for effective modeling and forecasting. The choice of method depends on the specific characteristics of the time series, such as the presence of trends or seasonality. Often, a combination of methods is used to achieve stationarity. After transforming the series, it's important to test for stationarity using statistical tests, such as the Augmented Dickey-Fuller test, to confirm that the transformation was successful.


### Auto Regressive Modeling
An AR model is a Linear Regression model, that uses lagged variables as input.
\begin{align}
        Y_t =C+b_1Y_{t-1}+b_2Y_{t-2}+…+b_pY_{t-p}+Er_t
    \end{align}

- $p$=past values
- $Y_t$=Function of different past values
- $Er_t$=errors in time
- $C$=intercept

```python
# implementing AR model
from statsmodels.tsa.ar_model import AutoReg

X = co2.average
train, test = X[1:len(X)-24], X[len(X)-24:]

# train autoregression
model = AutoReg(train, lags=20)
model_fit = model.fit()
print('Coefficients: %s' % model_fit.params)

# Predictions
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)

# plot results
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()
```

### ARMA (Autoregressive Moving Average)

The ARMA model is a cornerstone of time series analysis and combines two major components: autoregressive (AR) and moving average (MA). It's designed to model time series data that is stationary, meaning the series has a constant mean and variance over time, and its covariance is independent of the time at which the series is observed.

#### Components of ARMA:

- **Autoregressive (AR) part** ($p$): This component models the current value of the time series as a linear combination of its previous values. The parameter $p$ represents the order of the AR part, indicating the number of lagged observations included in the model.

\begin{align}
  AR(p): Y_t = \phi_1 Y_{t-1} + \phi_2 Y_{t-2} + \dots + \phi_p Y_{t-p} + \epsilon_t
\end{align}

- **Moving Average (MA) part** ($q$): This component models the current value of the series as a linear combination of the past error terms (the differences between past observations and predictions). The parameter $q$ represents the order of the MA part, indicating the number of lagged error terms included.
\begin{align}  
  MA(q): Y_t = \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \dots + \theta_q \epsilon_{t-q}
\end{align}
- **Combining AR and MA** gives the ARMA model, which can be written as:
\begin{align}
  ARMA(p, q): Y_t = \phi_1 Y_{t-1} + \dots + \phi_p Y_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_{t-q}
\end{align}

### ARIMA (Autoregressive Integrated Moving Average)

ARIMA extends the ARMA model to include integration (I), making it suitable for analyzing non-stationary time series. The integration part involves differencing the time series one or more times to make it stationary.

#### Components of ARIMA:

- **Autoregressive (AR) part** ($p$): Same as in ARMA, it models the current value as a function of its previous values.
- **Integrated (I) part** ($d$): This represents the order of differencing required to make the series stationary. Differencing is the process of subtracting the current observation from the previous observation. If $d$ differencing operations are required to achieve stationarity, the model is said to be integrated of order $d$.
- **Moving Average (MA) part** ($q$): As with ARMA, it models the current value using the past forecast errors.

The ARIMA model can thus be represented as ARIMA($p, d, q$), where:
- $p$ = order of the autoregressive part,
- $d$ = degree of first differencing involved,
- $q$ = order of the moving average part.

#### Formula:
\begin{align}
\Delta^d Y_t = \phi_1 \Delta^d Y_{t-1} + \dots + \phi_p \Delta^d Y_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_{t-q}
\end{align}
where $\Delta^d Y_t$ represents the $d$-times differenced series.

### Usage and Applications

- **ARMA** is best suited for stationary time series without trend and seasonal components.
- **ARIMA** is versatile and can handle a wide range of time series, including those with trends but not seasonal patterns.

For seasonal data, an extension of ARIMA known as Seasonal ARIMA (SARIMA) is often used, which incorporates seasonal differencing along with additional seasonal AR and MA terms.

These models are fundamental in forecasting, allowing for predictions based on past values and the errors associated with those past predictions. Correctly specifying the $p$, $d$, and $q$ parameters is crucial for the success of these models, often achieved through iterative testing and model selection criteria such as AIC (Akaike Information Criterion) or BIC (Bayesian Information Criterion).

### Example of ARIMA
```python
# ARIMA
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(co2.average, order=(10, 2, 3))
# the order parameters:
# p: No. of lag observations.
# d: No. of times that the raw observations are differenced.
# q: the size of the moving average window
results_ARIMA = model.fit()
results_ARIMA.summary()
results_ARIMA.plot_predict(start=700)
plt.show()
```